In [1]:
import ast
import nose
import sys
from unittest.mock import patch
from io import StringIO

In [2]:
q_pth = '../questions/q_foo.py'

In [3]:
with open(q_pth) as f:
    full_text_code = f.read() 

In [4]:
code_obj = compile(full_text_code, '<string>', 'exec')

In [5]:
exec(code_obj)

In [6]:
answer('hi')

True

In [7]:
user_answer = '''\
def answer(*foos):
    return all(f.startswith('foo') for f in foos)
'''

In [8]:
class StdIOWarpper:
    def __init__(self, buf):
        print('init with %r' % buf)
        self.buf = buf
    
    def __enter__(self):
        self.orig_stdout = sys.stdout
        self.orig_stderr = sys.stderr
        sys.stdout = sys.stderr = self.buf

    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout = self.orig_stdout
        sys.stderr = self.orig_stderr
        print(exc_type, exc_value, traceback)
        
    @property
    def captured(self):
        return self.buf.getvalue()

In [9]:
with patch('sys.stderr', new_callable=StringIO) as p:
    nose_config = nose.config.Config()
    nose_config.verbosity = 2
    nose.runmodule(argv=[''], exit=False, config=nose_config)
    print(p.getvalue())

__main__.test_answer_foo ... ok
__main__.test_answer_bar ... FAIL
__main__.test_hidden_large_list_foos ... FAIL

FAIL: __main__.test_answer_bar
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/Users/liang/.virtualenvs/pycode/lib/python3.4/site-packages/nose/case.py", line 198, in runTest
    self.test(*self.arg)
  File "<string>", line 24, in test_answer_bar
AssertionError

FAIL: __main__.test_hidden_large_list_foos
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/Users/liang/.virtualenvs/pycode/lib/python3.4/site-packages/nose/case.py", line 198, in runTest
    self.test(*self.arg)
  File "<string>", line 28, in test_hidden_large_list_foos
AssertionError

----------------------------------------------------------------------
Ran 3 tests in 0.001s

FAILED (failures=2)



In [10]:
class EmbedTestProgram(nose.core.TestProgram):
    def runTests(self):
        """Run Tests. Returns true on success, false on failure, and sets
        self.success to the same value.
        """
        if self.testRunner is None:
            self.testRunner = nose.core.TextTestRunner(stream=self.config.stream,
                                             verbosity=self.config.verbosity,
                                             config=self.config)
        plug_runner = self.config.plugins.prepareTestRunner(self.testRunner)
        if plug_runner is not None:
            self.testRunner = plug_runner
        self.result = self.testRunner.run(self.test)
        self.success = self.result.wasSuccessful()

In [11]:
def run_test():
    buf = StringIO()
    nose_config = nose.config.Config()
    nose_config.verbosity = 2
    nose_config.stream = buf
    test_prog = EmbedTestProgram(defaultTest='__main__', argv=[''], exit=False, config=nose_config)
    return test_prog, buf.getvalue()

In [12]:
tp, test_output = run_test()
tp.success

False

In [22]:
tp.result.errors

[]

In [23]:
tp.result.failures

[(Test(__main__.test_answer_bar),
  'Traceback (most recent call last):\n  File "/Users/liang/.virtualenvs/pycode/lib/python3.4/site-packages/nose/case.py", line 198, in runTest\n    self.test(*self.arg)\n  File "<string>", line 24, in test_answer_bar\nAssertionError\n'),
 (Test(__main__.test_hidden_large_list_foos),
  'Traceback (most recent call last):\n  File "/Users/liang/.virtualenvs/pycode/lib/python3.4/site-packages/nose/case.py", line 198, in runTest\n    self.test(*self.arg)\n  File "<string>", line 28, in test_hidden_large_list_foos\nAssertionError\n')]

In [44]:
tp.result.errors

[]

In [13]:
t = tp.result.failures[0]

In [14]:
t[0].address()

(None, '__main__', 'test_answer_bar')

In [21]:
print(t[1])

Traceback (most recent call last):
  File "/Users/liang/.virtualenvs/pycode/lib/python3.4/site-packages/nose/case.py", line 198, in runTest
    self.test(*self.arg)
  File "<string>", line 24, in test_answer_bar
AssertionError



In [16]:
user_answer_code = compile(user_answer, '<string>', 'exec')

In [17]:
exec(user_answer_code)

In [19]:
tp, test_output = run_test()
tp.success

True